<a href="https://colab.research.google.com/github/Pavitra-khare/DA6401_ASS3_withoutAtten-/blob/main/ASS3_WithoutAttenipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and login

In [ ]:
!pip install pytorch_lightning
!pip install wandb



In [ ]:
import wandb
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
import torch
import torch.nn.functional as F
torch.cuda.empty_cache()
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch.utils.data as data
import pandas as pd
import os
import csv
warnings.filterwarnings("ignore")

In [ ]:
wandb.login(key="71aebd5eed3e9b3e37a5a3c4658f5433375d97dc")


# Data Preprocessing

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

'''Location of your CSV file (Extracted file)
Location of your CSV file if on kaggle than zip file is fine'''

trainFilepath="/kaggle/input/myinput/aksharantar_sampled/hin/hin_train.csv"
valFilePath="/kaggle/input/myinput/aksharantar_sampled/hin/hin_valid.csv"
testFilePath="/kaggle/input/myinput/aksharantar_sampled/hin/hin_test.csv"

In [ ]:
def read_file_0(filepath):
    def process_row(text_row):
        return list(text_row)

    def collect_characters(reader_obj):
        all_chars = []
        for entry in reader_obj:
            all_chars += process_row(entry[0])
        return all_chars

    with open(filepath, mode='r') as file_handle:
        csv_reader = csv.reader(file_handle)
        return collect_characters(csv_reader)


In [ ]:

# trainFilepath = '/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_train.csv'


chars = read_file_0(trainFilepath)
setChar=set(chars)
setChar.add('|')
setOfchar = list(setChar)

# Create the association between characters and their corresponding integer indices
char_to_idx_latin= {char: i+1 for i, char in enumerate(setOfchar)}

In [ ]:
def read_file_1(trainFilepath):
    def extract_chars(text):
        return list(text)

    def accumulate(reader_obj):
        buffer = []
        for entry in reader_obj:
            buffer += extract_chars(entry[1])
        return buffer

    with open(trainFilepath, 'r') as file_stream:
        csv_rows = csv.reader(file_stream)
        return accumulate(csv_rows)

In [ ]:
maxLenDev=0

chars = read_file_1(trainFilepath)
setChar=set(chars)
setChar.add('|')
setOfchar = list(setChar)

charToIndLang ={char: i+1 for i, char in enumerate(setOfchar)}

In [ ]:
with open(trainFilepath, 'r') as csv_file:
    reader = csv.reader(csv_file)
    max_length = 0

    def update_max(current_max, candidate):
        return candidate if candidate > current_max else current_max

    for record in reader:
        length = len(record[0])
        max_length = update_max(max_length, length)

    maxLenEng = max_length

In [ ]:
with open(trainFilepath, 'r') as csv_stream:
    reader_obj = csv.reader(csv_stream)
    longest = 0

    def get_larger(a, b):
        return b if b > a else a

    for line in reader_obj:
        current_length = len(line[1])
        longest = get_larger(longest, current_length)

    maxLenDev = longest

characters in words -> indices

In [ ]:
def convert_characters_to_indices(word, dictionary):
    def safe_lookup(char):
        return dictionary[char] if char in dictionary else -1

    mapped = [safe_lookup(ch) for ch in word]
    filtered = list(filter(lambda idx: idx >= 0, mapped))
    return filtered

In [ ]:
def adjust_sequence_length(indices, maximumLength):
    current_length = len(indices)

    def trim(seq, length):
        return seq[:length]

    def pad(seq, total_length):
        padding_needed = total_length - len(seq)
        return seq + [0] * padding_needed

    if current_length > maximumLength:
        return trim(indices, maximumLength)
    elif current_length < maximumLength:
        return pad(indices, maximumLength)
    return indices


In [ ]:
def convert_indices_to_tensor(indices, dictionary):
    token = dictionary.get('|', 0)

    def add_delimiters(seq, token_id):
        return [token_id] + seq + [token_id]

    sequence = add_delimiters(indices, token)
    tensor_obj = torch.tensor(sequence, device=device)
    return tensor_obj

In [ ]:
def convert_word_to_indices(word, maximumLength, dict):
    char_ids = convert_characters_to_indices(word, dict)

    def standardize_length(seq, target_len):
        return adjust_sequence_length(seq, target_len)

    resized = standardize_length(char_ids, maximumLength)
    tensor_out = convert_indices_to_tensor(resized, dict)
    return tensor_out


In [ ]:
def assign_tensor_to_generated_sequences(sequence):
    tokens = sequence.split()

    def concatenate(tokens):
        return ''.join(tokens)

    def compute_total_length(tokens):
        return sum(len(token) for token in tokens)

    combined = concatenate(tokens)
    total_len = compute_total_length(tokens)

    return combined, total_len

In [ ]:
def assemble_tensor(final_tensor, partition_size=1):
    def safe_partition_size(size):
        return max(1, size)

    chunk_size = safe_partition_size(partition_size)
    segments = [final_tensor[idx:idx + chunk_size] for idx in range(0, len(final_tensor), chunk_size)]

    return segments

In [ ]:
def assemble_assigned_generated_seq(path):
    combined_seq, total_chars = assign_tensor_to_generated_sequences(path)

    def determine_chunk_size(length, divisor=4):
        return max(1, length // divisor)

    segment_length = determine_chunk_size(total_chars)
    partitioned = assemble_tensor(combined_seq, segment_length)

    return partitioned


In [ ]:
def generate_indices(row):
    src_text = row[0]
    tgt_text = row[1]

    def build_indexed_tensor(text, max_len, char_map):
        return convert_word_to_indices(text, max_len, char_map)

    source_tensor = build_indexed_tensor(src_text, maxLenEng, char_to_idx_latin)
    target_tensor = build_indexed_tensor(tgt_text, maxLenDev, charToIndLang)

    return source_tensor, target_tensor


In [ ]:
pairs_v = []

with open(valFilePath, 'r') as val_file:
    csv_reader = csv.reader(val_file)

    def process_and_append(pairs, record):
        eng_tensor, hin_tensor = generate_indices(record)
        pairs.append([eng_tensor, hin_tensor])

    for entry in csv_reader:
        process_and_append(pairs_v, entry)


In [ ]:
pairs_t = []

with open(testFilePath, 'r') as test_file:
    test_reader = csv.reader(test_file)

    def append_indexed_pair(container, data_row):
        src_tensor, tgt_tensor = generate_indices(data_row)
        container.append([src_tensor, tgt_tensor])

    for record in test_reader:
        append_indexed_pair(pairs_t, record)


In [ ]:
pairs = []

with open(trainFilepath, 'r') as train_file:
    train_reader = csv.reader(train_file)

    def process_row_and_store(storage, row_data):
        input_tensor, target_tensor = generate_indices(row_data)
        storage.append([input_tensor, target_tensor])

    for entry in train_reader:
        process_row_and_store(pairs, entry)


#Get Dataloaders

In [ ]:
batch_size = 32
train_shuffle = True
eval_shuffle = False

def create_loader(dataset, size, shuffle_flag):
    return torch.utils.data.DataLoader(dataset, batch_size=size, shuffle=shuffle_flag)

dataloaderTrain = create_loader(pairs, batch_size, train_shuffle)
dataloaderVal = create_loader(pairs_v, batch_size, eval_shuffle)
dataloaderTest = create_loader(pairs_t, batch_size, eval_shuffle)


#CLASS FOR ENCODER DECODER

In [ ]:
import torch
import torch.nn as nn

# Encoder class (no attention mechanism)
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embed_size, cell_type, dropout_rate, num_layers, is_bidirectional):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.is_bidirectional = is_bidirectional

        self.embed_layer = nn.Embedding(input_dim, embed_size)

        rnn_args = {
            "input_size": embed_size,
            "hidden_size": hidden_dim,
            "dropout": dropout_rate,
            "num_layers": num_layers,
            "bidirectional": is_bidirectional
        }

        if cell_type == "GRU":
            self.rnn = nn.GRU(**rnn_args)
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(**rnn_args)
        elif cell_type == "RNN":
            self.rnn = nn.RNN(**rnn_args)
        else:
            raise ValueError(f"Unsupported cell type: {cell_type}")

    def forward(self, input_seq):
        embedded_seq = self.embed_layer(input_seq)
        output, hidden_state = self.rnn(embedded_seq)
        return hidden_state


# Decoder class (no attention mechanism)
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embed_size, cell_type, dropout_rate, num_layers, is_bidirectional):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.cell_type = cell_type
        self.is_bidirectional = is_bidirectional

        direction_factor = 2 if is_bidirectional else 1

        self.embed_layer = nn.Embedding(output_dim, embed_size)

        rnn_args = {
            "input_size": embed_size,
            "hidden_size": hidden_dim,
            "dropout": dropout_rate,
            "num_layers": num_layers,
            "bidirectional": is_bidirectional
        }

        if cell_type == "GRU":
            self.rnn = nn.GRU(**rnn_args)
        elif cell_type == "RNN":
            self.rnn = nn.RNN(**rnn_args)
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(**rnn_args)
        else:
            raise ValueError(f"Unsupported cell type: {cell_type}")

        self.output_layer = nn.Linear(hidden_dim * direction_factor, output_dim)

    def forward(self, input_token, hidden_state):
        embedded = self.embed_layer(input_token.unsqueeze(0))  # shape: (1, batch, embed_size)
        rnn_output, updated_hidden = self.rnn(embedded, hidden_state)
        logits = self.output_layer(rnn_output.squeeze(0))  # remove time dimension
        return logits, updated_hidden


#SEQ2SEQ class

In [ ]:
class Seq2Seq(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden_size, embed_size, rnn_type, dropout, enc_layers, dec_layers, is_bidirectional, lr,char_to_idx_latin, charToIndLang):
        super().__init__()
        self.lr = lr
        self.rnn_type = rnn_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.is_bidirectional = is_bidirectional
        self.dir_mult = 2 if is_bidirectional else 1
        self.char2lat = char_to_idx_latin   # source vocab
        self.char2hin = charToIndLang       # target vocab

        # Initialize encoder and decoder
        self.encoder = Encoder(input_dim, hidden_size, embed_size, rnn_type, dropout, enc_layers, is_bidirectional)
        self.decoder = Decoder(output_dim, hidden_size, embed_size, rnn_type, dropout, dec_layers, is_bidirectional)

        # Logging containers for metrics
        self.train_loss_log, self.train_acc_log = [], []
        self.val_loss_log, self.val_acc_log = [], []
        self.test_loss_log, self.test_acc_log = [], []

    def forward(self, src_seq, tgt_seq, teacher_prob=0.5):
        # Sequence-to-sequence forward pass with teacher forcing
        batch_size, seq_len = tgt_seq.shape
        vocab_size = self.decoder.output_layer.out_features
        prediction_store = torch.zeros(seq_len, batch_size, vocab_size).to(self.device)

        src_seq = src_seq.transpose(0, 1)
        hidden_state = self._fix_hidden_dims(self.encoder(src_seq))

        current_token = tgt_seq[:, 0]
        for step in range(1, seq_len):
            pred_output, hidden_state = self.decoder(current_token, hidden_state)
            prediction_store[step] = pred_output
            current_token = pred_output.argmax(1) if teacher_prob < torch.rand(1).item() else tgt_seq[:, step]

        return prediction_store

    def _fix_hidden_dims(self, hidden):
        # Adjust hidden layers if encoder and decoder layer counts mismatch
        lstm_mode = self.rnn_type == "LSTM"
        enc_more = self.enc_layers > self.dec_layers
        dec_more = self.enc_layers < self.dec_layers

        if enc_more:
            cut = (self.enc_layers - self.dec_layers) * self.dir_mult
            if lstm_mode:
                h, c = hidden
                return (h[cut:], c[cut:])
            return hidden[cut:]

        if dec_more:
            add = self.dec_layers - self.enc_layers
            if lstm_mode:
                h, c = hidden
                h_last, c_last = h[-self.dir_mult:], c[-self.dir_mult:]
                h = torch.cat([h] + [h_last] * add, dim=0)
                c = torch.cat([c] + [c_last] * add, dim=0)
                return (h, c)
            else:
                h_last = hidden[-self.dir_mult:]
                return torch.cat([hidden] + [h_last] * add, dim=0)

        return hidden

    def forward_pass(self, src, tgt):
        # Wraps model forward for clarity
        return self(src, tgt)

    def compute_expected_values(self, preds, target):
        # Create one-hot encoded expected output tensor
        result = torch.zeros_like(preds)
        result[torch.arange(preds.shape[0]), torch.arange(preds.shape[1]).unsqueeze(1), target.cpu()] = 1
        return result

    def calculate_loss(self, preds, expected, target):
        # Cross-entropy loss computation (skip BOS)
        d = preds.shape[-1]
        return self.loss_fn(preds[1:].reshape(-1, d), expected[1:].reshape(-1, d))

    def calculate_accuracy(self, preds, target):
        # Accuracy at word level (after permute)
        preds = preds.permute(1, 0, 2)
        return self._wordwise_accuracy(preds, target)

    def update_metrics(self, loss_val, acc_val):
        # Append training metrics to logs
        self.train_loss_log.append(loss_val.detach())
        self.train_acc_log.append(torch.tensor(acc_val))

    def training_step(self, batch, batch_idx):
        # Called during each training batch
        src, tgt = batch
        logits = self.forward_pass(src, tgt)
        expected = self.compute_expected_values(logits, tgt)
        loss = self.calculate_loss(logits, expected, tgt)
        acc = self.calculate_accuracy(logits, tgt)
        self.update_metrics(loss, acc)
        return {'loss': loss}

    def forward_pass_validation(self, src, tgt):
        # Run forward without teacher forcing
        return self(src, tgt, teacher_prob=0.0)

    def validation_step(self, batch, batch_idx):
        # Called for each validation batch
        src, tgt = batch
        logits = self.forward_pass_validation(src, tgt)
        expected = self.compute_expected_values(logits, tgt)
        loss = self.calculate_loss(logits, expected, tgt)
        acc = self.calculate_accuracy(logits, tgt)
        self.val_loss_log.append(loss.detach())
        self.val_acc_log.append(torch.tensor(acc))
        return {'loss': loss}

    def test_step(self, batch, batch_idx):
        """
        One full forward pass on the test set (no teacher forcing).
        Computes and logs loss and word-level accuracy, and saves predictions.
        """
        src_seq, tgt_seq = batch
        logits = self(src_seq, tgt_seq, teacher_prob=0.0)
        gold_one_hot = self.compute_expected_values(logits, tgt_seq)
        loss_val = self.calculate_loss(logits, gold_one_hot, tgt_seq)
        acc_val = self._wordwise_accuracy(logits.permute(1, 0, 2), tgt_seq)

        # Save predictions to CSV
        src_tokens, tgt_tokens, pred_tokens = self.grid(src_seq, logits.permute(1, 0, 2), tgt_seq)

        def tokens_to_string(token_list, vocab):
            return "".join([k for tok in token_list for k, v in vocab.items() if v == tok])

        inputs = [tokens_to_string(seq.tolist(), self.char2lat) for seq in src_tokens]
        targets = [tokens_to_string(seq.tolist(), self.char2hin) for seq in tgt_tokens]
        predictions = [tokens_to_string(seq.tolist(), self.char2hin) for seq in pred_tokens]

        save_outputs_to_csv(inputs, targets, predictions)

        #uncomment below code to log predictions on test data
        # log_predictions_to_wandb(inputs, targets, predictions)

        self.test_loss_log.append(loss_val.detach())
        self.test_acc_log.append(torch.tensor(acc_val))
        return {"loss": loss_val}

    def on_test_epoch_end(self):
        # Print and log test epoch results
        loss_avg = torch.stack(self.test_loss_log).mean()
        acc_avg = torch.stack(self.test_acc_log).mean()
        self.test_loss_log.clear()
        self.test_acc_log.clear()
        print({"test_loss": loss_avg.item(), "testAccuracy": acc_avg.item()})
        wandb.log({"test_loss_last": loss_avg, "testAccuracy_last": acc_avg})

    def on_train_epoch_end(self):
        # Print and log training & validation metrics
        tr_loss = torch.stack(self.train_loss_log).mean()
        val_loss = torch.stack(self.val_loss_log).mean()
        tr_acc = torch.stack(self.train_acc_log).mean()
        val_acc = torch.stack(self.val_acc_log).mean()

        self.train_loss_log.clear()
        self.val_loss_log.clear()
        self.train_acc_log.clear()
        self.val_acc_log.clear()

        print({
            "Train Loss": round(tr_loss.item(), 3),
            "Train Accuracy": round(tr_acc.item(), 3),
            "Validation Loss": round(val_loss.item(), 3),
            "Validation Accuracy": round(val_acc.item(), 3)
        })
        wandb.log({
            "Train Loss": tr_loss,
            "Train Accuracy": tr_acc,
            "Validation Loss": val_loss,
            "Validation Accuracy": val_acc
        })

    def configure_optimizers(self):
        # Adam optimizer setup
        return optim.Adam(self.parameters(), lr=self.lr)

    def loss_fn(self, out, target):
        # Cross-entropy loss wrapper
        return nn.CrossEntropyLoss()(out, target).mean()

    def _wordwise_accuracy(self, logits, gold):
        # Computes word-level match accuracy
        preds = logits.argmax(dim=-1)
        match_count = sum(torch.equal(gold[i, 1:-1], preds[i, 1:-1]) for i in range(gold.size(0)))
        return (match_count / gold.size(0)) * 100

    def grid(self, inputs, outputs, targets):
        # Converts batch output into src-tgt-predicted token lists
        guesses = outputs.argmax(dim=-1)
        src_tokens, tgt_tokens, pred_tokens = [], [], []
        for idx in range(targets.size(0)):
            src_tokens.append(inputs[idx, 1:-1])
            tgt_tokens.append(targets[idx, 1:-1])
            pred_tokens.append(guesses[idx, 1:-1])
        return src_tokens, tgt_tokens, pred_tokens


In [ ]:

def save_outputs_to_csv(inputs, targets, predictions):
    output_file = 'Output.csv'
    already_present = os.path.isfile(output_file)

    records = {
        'input': inputs,
        'target': targets,
        'predicted': predictions
    }

    dataframe = pd.DataFrame(records)
    dataframe.to_csv(output_file, mode='a', index=False, header=not already_present)


def log_predictions_to_wandb(inputs, targets, preds):
    prediction_table = wandb.Table(columns=["Input", "Target", "Prediction Result"])

    for src, gold, guess in zip(inputs, targets, preds):
        result = "✅" if gold == guess else "❌"
        labeled_pred = f"{guess} {result}"
        prediction_table.add_data(src, gold, labeled_pred)

    wandb.log({"Predictions Overview": prediction_table})







# function will return key for given value
def index_to_char_latin(index):
    return next((char for char, idx in char_to_idx_latin.items() if idx == index), "")

def index_to_char_lang(index):
    return next((char for char, idx in charToIndLang.items() if idx == index), "")



#SWEEP CONFIG

In [ ]:
sweep_config_NOAttn = {
    # Bayesian Search for hyperparameters
    "name" : "Bayesian Sweep",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "Validation Accuracy"},
    "parameters": {'drop_out': {"values": [0.3,0.5]},
                   'embeddingSize': {"values": [64,128,256]},
                   'hidden_layer_size': {"values": [128,256,512]},
                   'layersEncoder': {"values": [2, 3]},
                   'layersDecoder': {"values": [2, 3]},
                   "cellType": {"values": [ "RNN", "GRU", "LSTM"]},
                   "learningRate": {"values": [1e-3, 0.005]},
                   "bidirectional":{"values":[True, False]},
                   "epochs": {"values": [10, 15]}
                }
}

In [ ]:
def train_no_attn():
    run = wandb.init(project="DA6401_ASS3_VANILLA", name="test_VANILLA_grid")
    config = run.config

    # Set a descriptive name for this specific run
    wandb.run.name = (
        f"_cell_{config.cellType}_dr_{config.drop_out}"
        f"_em_{config.embeddingSize}_hl_{config.hidden_layer_size}"
        f"_en_{config.layersEncoder}_de_{config.layersDecoder}"
        f"_lr_{config.learningRate}_bi_{config.bidirectional}"
        f"_ep_{config.epochs}"
    )

    # Extract hyperparameters
    model_params = {
        "input_dim": len(char_to_idx_latin) + 2,
        "output_dim": len(charToIndLang) + 2,
        "hidden_dim": config.hidden_layer_size,
        "embedding_dim": config.embeddingSize,
        "cell_type": config.cellType,
        "dropout": config.drop_out,
        "enc_layers": config.layersEncoder,
        "dec_layers": config.layersDecoder,
        "bidirectional": config.bidirectional,
        "lr": config.learningRate
    }

    # Model initialization (no attention)
    model = Seq2Seq(
        model_params["input_dim"],
        model_params["output_dim"],
        model_params["hidden_dim"],
        model_params["embedding_dim"],
        model_params["cell_type"],
        model_params["dropout"],
        model_params["enc_layers"],
        model_params["dec_layers"],
        model_params["bidirectional"],
        model_params["lr"]
    )

    model.to(device)

    # Train the model using PyTorch Lightning
    trainer = pl.Trainer(
        max_epochs=config.epochs,
        accelerator="gpu",
        devices=1
    )
    trainer.fit(model, train_dataloaders=dataloaderTrain, val_dataloaders=dataloaderVal)


In [ ]:
# # Running Sweeps for no attention
sweep_id = wandb.sweep(sweep_config_NOAttn, project='DA6401_ASS3_VANILLA')
wandb.agent(sweep_id, train_no_attn, count = 40)
wandb.finish()

#BEST PARAMS
